In [141]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import string
import nltk

In [111]:
df1 = pd.read_csv('/content/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('/content/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('/content/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')

In [112]:
df1.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [113]:
df1.shape

(1715, 6)

In [114]:
df2.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,S09_set4_a10
1,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,S09_set4_a10
2,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,S09_set4_a10
3,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,S09_set4_a10
4,Alessandro_Volta,Did Volta have a passion for the study of elec...,yes,easy,medium,S09_set4_a10


In [115]:
df3.shape

(1458, 6)

In [116]:
df3.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,S10_set4_a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,S10_set4_a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,S10_set4_a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,S10_set4_a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,S10_set4_a10


In [117]:
df3.shape

(1458, 6)

In [118]:
df=df1.append([df2,df3])

<ipython-input-118-b7b12840a892>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append([df2,df3])


In [119]:
df.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [120]:
df.shape

(3998, 6)

# Clean Data and Merge

In [121]:
df['Question'] = df['ArticleTitle'].str.replace('_', ' ') + ' ' + df['Question']
all_data = df[['Question', 'Answer']]
all_data.shape

(3998, 2)

In [122]:
all_data.head()

,Question,Answer
0,Abraham Lincoln Was Abraham Lincoln the sixtee...,yes
1,Abraham Lincoln Was Abraham Lincoln the sixtee...,Yes.
2,Abraham Lincoln Did Lincoln sign the National ...,yes
3,Abraham Lincoln Did Lincoln sign the National ...,Yes.
4,Abraham Lincoln Did his mother die of pneumonia?,no


In [123]:
all_data.shape

(3998, 2)

In [124]:
all_data.head(10)['Question']

0    Abraham Lincoln Was Abraham Lincoln the sixtee...
1    Abraham Lincoln Was Abraham Lincoln the sixtee...
2    Abraham Lincoln Did Lincoln sign the National ...
3    Abraham Lincoln Did Lincoln sign the National ...
4     Abraham Lincoln Did his mother die of pneumonia?
5     Abraham Lincoln Did his mother die of pneumonia?
6    Abraham Lincoln How many long was Lincoln's fo...
7    Abraham Lincoln How many long was Lincoln's fo...
8    Abraham Lincoln When did Lincoln begin his pol...
9    Abraham Lincoln When did Lincoln begin his pol...
Name: Question, dtype: object

In [125]:
all_data=all_data.drop_duplicates(subset='Question')

In [126]:
all_data

,Question,Answer
0,Abraham Lincoln Was Abraham Lincoln the sixtee...,yes
2,Abraham Lincoln Did Lincoln sign the National ...,yes
4,Abraham Lincoln Did his mother die of pneumonia?,no
6,Abraham Lincoln How many long was Lincoln's fo...,18 months
8,Abraham Lincoln When did Lincoln begin his pol...,1832
...,...,...
1448,Zebra What do zebras eat?,mainly grass
1450,Zebra What are zebras hunted for?,mainly for their skins
1452,Zebra What areas do the Grevy's Zebras inhabit?,semi-arid grasslands of Ethiopia and northern ...
1454,Zebra Which species of zebra is known as the c...,"Plains Zebra (Equus quagga, formerly Equus bur..."


In [127]:
all_data.shape

(2461, 2)

In [128]:
df=all_data.dropna()

In [129]:
df.shape

(2188, 2)

In [130]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [131]:
stopwords_list=stopwords.words('english')

In [132]:
stopwords_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [133]:
df.head()

,Question,Answer
0,Abraham Lincoln Was Abraham Lincoln the sixtee...,yes
2,Abraham Lincoln Did Lincoln sign the National ...,yes
4,Abraham Lincoln Did his mother die of pneumonia?,no
6,Abraham Lincoln How many long was Lincoln's fo...,18 months
8,Abraham Lincoln When did Lincoln begin his pol...,1832


In [136]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [143]:
stopwords_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def tokenizer(doc):
    words = word_tokenize(doc)

    pos_tags = pos_tag(words)

    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]

    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]

    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN

        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas


tfidf = TfidfVectorizer(tokenizer=tokenizer)
tfidf_matrix = tfidf.fit_transform(tuple(df['Question']))

In [145]:
tfidf_matrix.shape

(2188, 3550)

In [146]:
def ask_question(question):
  query_vect=tfidf.transform([question])
  similarity=cosine_similarity(query_vect,tfidf_matrix)
  max_similarity=np.argmax(similarity,axis=None)




  print('Your Question',question)
  print('Closest question found:', all_data.iloc[max_similarity]['Question'])
  print('Similarity: {:.2%}'.format(similarity[0, max_similarity]))
  print('Answer:', all_data.iloc[max_similarity]['Answer'])

In [147]:
ask_question('When Abraham Lincoln started his political career')

Your Question When Abraham Lincoln started his political career
Closest question found: Abraham Lincoln Did Lincoln start his political career in 1832?
Similarity: 88.14%
Answer: Yes


In [148]:
ask_question('Where was Nicola Tesla born')

Your Question Where was Nicola Tesla born
Closest question found: Abraham Lincoln Which county was Lincoln born in?
Similarity: 40.56%
Answer: Hardin County
